In [23]:
# Imports
import SDL
from SDL.utility import *

First, a "campaign" or list of campaigns will be created. A campaign consists of an instance of SDL.utility.CampaignInfo. It is a vessel for carrying all information necessary to perform a set of (V)SDL runs. It begines with building an environment. To do so, declare the relevant experiments instances from SDL.world. Check that module for the current options. Then, decide which environment to build. Current option is only VSDLEnvironment. Then, the ML agent will ne to be declared. Decide which to use from SDL.strategy, then feed it the necessary arguments (if deviating from defaults). Finally, you are ready to put it all together into a CampaignInfo instance. You can adjust some campaign arguments, like the number of runs to perform, the sampling procedure, the campaign name, etc.

In [24]:
# Create the experiments
experiments = {
    'BladeCoat': SDL.world.BladeCoat(action_space=['Stability'],inputs={'temperature':[0,400], 'time':[0,300]},cost=10.0),
    'Stability': SDL.world.Stability(cost=10.0),
}

# Create the environment
environment = SDL.world.VSDLEnvironment(experiments=experiments)

# Create the agent
agent = SDL.strategy.ArchitectureOne(epsilon=0.3)

# Combine into a campagin
campaign1 = CampaignInfo(
    name = 'ArchitectureZero -- Environment 000',
    runs = 1,
    environment = environment,
    agent = agent,
    sampling_procedure = [
        ('number_of_initial_datapoints', 3),
        ('number_of_batches', 10),
        ('samples_per_batch', 2)
        ],
)

# Create another campaign
experiments = {
    'BladeCoat': SDL.world.BladeCoat(action_space=['Stability'],inputs={'temperature':[0,400], 'time':[0,300]},cost=10.0),
    'Stability': SDL.world.Stability(cost=10.0),
}
environment = SDL.world.VSDLEnvironment(experiments=experiments)
agent = SDL.strategy.ArchitectureOne(epsilon=0.5)
campaign2 = CampaignInfo(
    name = 'ArchitectureZero -- Environment 001',
    runs = 1,
    environment = environment,
    agent = agent,
    sampling_procedure = [
        ('number_of_initial_datapoints', 4),
        ('number_of_batches', 1),
        ('samples_per_batch', 3)
        ],
)

# Compile into a campaign list
campaign_list = [campaign1]#,campaign2]

Next, the campaigns are run. If the number of campaigns and the number of samples to be performed are small, they can be run right in this notebook. If there are many samples, many runs, and/or many campaigns, consider running them using HPC. To do this, dump the campaign list into a binary file using pickle. Then, use the runSDL.py module in this repo to read and run the binary file.

In [25]:
# Run campaign in notebook
#for _ in campaign_list:
#    _.run()

# Dump to a binary input file, to be run on an HPC
dump_campaign_list(campaign_list, 'my_VSDL_inputs {} .pkl'.format(datetime.datetime.now()))

Next, read the output file (and optionally the MAE file) generated from a campaign. The data will be read in as an instance of SDL.utility.SDLOutputData.

In [11]:
# Read in output from a campaign
data = read_output('ArchitectureZero -- Environment 000.out.txt')
for k,v in data.__dict__.items():
    print(k,v)

name ArchitectureZero -- Environment 000
color #
calc_stability None
runs 1
sampling_procedure [('Initial Datapoints per Processing Variable', 3), ('Number of Batches', 10), ('Samples per Batch', 2)]
states {1: [[[<SDL.material.State object at 0x7fe160781b20>, <SDL.material.State object at 0x7fe160781d00>], [<SDL.material.State object at 0x7fe1408a7250>, <SDL.material.State object at 0x7fe1607728b0>], [<SDL.material.State object at 0x7fe160772d00>, <SDL.material.State object at 0x7fe1607927c0>], [<SDL.material.State object at 0x7fe1607924c0>, <SDL.material.State object at 0x7fe148db64c0>], [<SDL.material.State object at 0x7fe1607875b0>, <SDL.material.State object at 0x7fe160787a90>], [<SDL.material.State object at 0x7fe16078ce50>, <SDL.material.State object at 0x7fe170b50bb0>], [<SDL.material.State object at 0x7fe160873310>, <SDL.material.State object at 0x7fe160873d00>], [<SDL.material.State object at 0x7fe148f29670>, <SDL.material.State object at 0x7fe1609d7cd0>], [<SDL.material.Stat

In [19]:
blade_coat = SDL.world.BladeCoat(inputs={'temperature': [0, 400], 'time': [0, 300]})
reference_inputs = blade_coat.get_input_space()[1]
reference_targets = SDL.world.g1(reference_inputs[:,0]/400,p2=reference_inputs[:,1]/300)
cumulative_MAEs = np.array([
        get_cumulative_MAE(data,run,reference_inputs,reference_targets)
            for run in range(1,data.runs+1)])

In [22]:
cumulative_MAEs

array([[20.03307681, 73.09567002, 20.13515907, 73.08970196, 17.20177432,
        73.06187483, 16.38559755, 16.38559767, 73.04039472, 16.33327455,
        16.33327467]])